In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler,LabelEncoder

import tensorflow as tf
from keras.layers import (Dense,Conv1D,BatchNormalization,GlobalAveragePooling1D
                          ,MaxPooling1D,AveragePooling1D,Multiply,Add)


from tensorflow.keras.models import model_from_json
from define_CNN_class import MyResNetModel
from define_CNN_class import ResidualBlock
from define_CNN_class import NoiseGate


ニューラルネットワークを宣言

In [ ]:
custom_objects={
        'MyResNetModel': MyResNetModel,
        'ResidualBlock':ResidualBlock,
        'NoiseGate':NoiseGate        
        }

with open(r'cnn_model_structual_data.json', 'r') as json_file:

    json_string = json_file.read()
    model = model_from_json(json_string,custom_objects=custom_objects)


model.load_weights(r"cnn_model.weights.h5")
model.summary()
model.compile(optimizer='adam', loss='mse')



In [ ]:
df1= pd.read_excel(r"PKSP_Q84mse(25,40,55)v2.xlsx",sheet_name="解析用平均値",header=1)
df1_X=df1[df1['Temp'] == 298]
df1_X=df1_X[df1['サンプルNo.'] >=17]
NMR_columns=df1_X.columns[10:320]

NMR_data=df1_X.copy().loc[:,NMR_columns]

for i in range(len(NMR_data)):
    data=NMR_data.iloc[i,:].copy()
    NMR_data.iloc[i,:]= (data-np.min(data))/(np.max(data)-np.min(data))
    #Nomalized_data.iloc[i,:]= data/np.max(data)

    #print(np.max(Nomalized_data.iloc[i,:]))
    #print(np.min(Nomalized_data.iloc[i,:]))

process_columns=df1_X.columns[2:6]#成型条件が格納されているcolumnを取得
process_data=df1_X.copy().loc[:,process_columns]#columnをもとにNMRデータを取得

#print(NMR_data)

index=[]
for i in range(len(NMR_data)):
    index.append(f"Q{i+17}-MSE-298-1")#サンプル番号、パルス名、温度、N番号

NMR_data.index=index
#print(NMR_data)

physics_data=pd.read_csv(r"物性のデータ.csv").loc[:,"y1":]
physics_data.index=index
process_data.index=index
print(physics_data)
print(process_data)

for i in range(4):
    le=LabelEncoder()
    le.fit(process_data.iloc[:,i])
    process_data.iloc[:,i]=le.transform(process_data.iloc[:,i])
    print(np.max( process_data.iloc[:,i]))

print(process_data)


In [ ]:
pred=[]
soft_noise=[]

CNN_smooth=pd.DataFrame()
CNN_gap=pd.DataFrame()
ave_smooth=pd.DataFrame()

for i in range(len(NMR_data)):
    #data=model.predict(average_smoothing(np.array(selected_MAPE_data.loc[f"Q1{i}-mape-298-1",:]).reshape(1,311,1),num=10)).reshape(-1)
    pred=model.predict(np.array(NMR_data.iloc[i,:]).reshape(1,310,1))
                       
    data,gap=pred[0].reshape(-1),pred[1].reshape(-1)#出力を取得
    CNN_smooth[f"Q{i+17}-MSE-298-1"]=data
    CNN_gap[f"Q{i+17}-MSE-298-1"]=gap
    #サンプル番号、パルス名、温度、N番号をラベルにする

    
CNN_smooth=CNN_smooth.T
CNN_gap=CNN_gap.T

CNN_gap.columns=[f"gap_{i}" for i in range(len(CNN_gap.columns))]
#ave_smooth=ave_smooth.T

CNN_smooth.columns=columns=np.arange(310)*0.00064

print(CNN_smooth.shape)
print(CNN_smooth)


print(CNN_gap.shape)
print(CNN_gap)


In [ ]:
SS_X=StandardScaler()
ss_physics_data=pd.DataFrame(SS_X.fit_transform(physics_data),index=physics_data.index,columns=physics_data.columns)

SS_X=StandardScaler()
ss_CNN_gap=pd.DataFrame(SS_X.fit_transform(CNN_gap),index=CNN_gap.index,columns=CNN_gap.columns)


process_physics_gap=pd.concat([process_data,ss_physics_data,ss_CNN_gap],axis=1)
print(process_physics_gap)

In [ ]:
df1= pd.read_excel(r"PKSP_Q84mse(25,40,55)v2.xlsx",sheet_name="解析用平均値",header=1)
df1_X=df1[df1['Temp'] == 298]
df1_X=df1_X[df1['サンプルNo.'] >=17]
NMR_columns=df1_X.columns[10:320]#NMRデータが格納されているcolumnを取得
NMR_data=df1_X.copy().loc[:,NMR_columns]#columnをもとにNMRデータを取得

for i in range(len(NMR_data)):
    data=NMR_data.iloc[i,:].copy()#安全のため一行ずつコピーを生成
    CNN_smooth.iloc[i,:]=(CNN_smooth.iloc[i,:].copy()*((np.max(data)-np.min(data))))+np.min(data)#正規化されていたデータを元に戻す



#print(NMR_data)
print(CNN_smooth)


In [ ]:
for i in range(len(NMR_data)):
    plt.plot(figsize=(100, 60))
    plt.title(f"denoising No.{i+1}")

    plt.plot(NMR_data.columns, NMR_data.iloc[i,:], label='raw_data')
    plt.plot(NMR_data.columns,  CNN_smooth.iloc[i,:], label='predict',color="black")

    plt.legend()
    plt.show()

    plt.plot(figsize=(100, 60))
    plt.title(f"logged denoising No.{i+1}")

    plt.plot(NMR_data.columns, np.log(NMR_data.iloc[i,:]), label='raw_data')
    plt.plot(NMR_data.columns,  np.log(CNN_smooth.iloc[i,:]), label='predict',color="black")

    plt.legend()
    plt.show()
    break

In [ ]:

print(np.array(CNN_gap.iloc[:,0]).reshape(-1).shape)
print(physics_data[f"y1"].shape)

SS_X=StandardScaler()
SS_X.fit((CNN_gap))
ss_gap=pd.DataFrame(SS_X.transform((CNN_gap)))


target_gap_list=[]
target_gap_num_list=[]

for i in range(1,10,1):
    print(f"y{i}")
    max_score=0
    index_num=0
    SS_Y=StandardScaler()
    SS_Y.fit(np.array(physics_data[f"y{i}"]).reshape(-1,1))
    y_data=SS_Y.transform(np.array(physics_data[f"y{i}"]).reshape(-1,1))
    
    

    for j in range(256):
        score=np.corrcoef(np.array(ss_gap.iloc[:,j]).reshape(-1),y_data.reshape(-1))[0,1]
        if abs(score)>max_score:
            max_score=(score)
            index_num=j
      

    print(index_num)
    print(max_score)#相関係数の最大値を表示
    
    plt.rcParams["font.size"]=20
    plt.rcParams["font.family"]="Arial"

    plt.figure(dpi=600)
    target_gap_list.append(f"gap_{index_num}")
    target_gap_num_list.append(index_num)

    for k in range(16):
        plt.scatter(ss_gap.iloc[0*16+k,index_num],y_data[0*16+k],c="blue")
        plt.scatter(ss_gap.iloc[1*16+k,index_num],y_data[1*16+k],c="green")
        plt.scatter(ss_gap.iloc[2*16+k,index_num],y_data[2*16+k],c="orange")
        plt.scatter(ss_gap.iloc[3*16+k,index_num],y_data[3*16+k],c="red")
    
    plt.xlabel("GAP")
    plt.ylabel("y")
    plt.show()
    
print(target_gap_list)
    

In [ ]:
SS_X=StandardScaler()
ss_physics_data=pd.DataFrame(SS_X.fit_transform(physics_data),index=physics_data.index,columns=physics_data.columns)

SS_X=StandardScaler()
ss_CNN_gap=pd.DataFrame(SS_X.fit_transform(CNN_gap),index=CNN_gap.index,columns=CNN_gap.columns)


process_physics_gap=pd.concat([process_data,ss_physics_data,ss_CNN_gap],axis=1)
print(process_physics_gap)

In [ ]:
data=np.array(NMR_data)
print(data.shape)
feature_map = model.predict(data.reshape(-1,310,1))[2]
print(feature_map.shape)




for i in range(len(target_gap_num_list)):
    print(f"y{i+1}")
    print(f"gap_{target_gap_num_list[i]}")
    
    print(np.sum(feature_map[0,:,target_gap_num_list[i]]))
    print(np.sum(feature_map[0,21:,target_gap_num_list[i]]))

    plt.figure(dpi=600)
    sns.heatmap(((feature_map[:,:,target_gap_num_list[i]])),vmin=-0.05,vmax=0.05)

    plt.show()
    
